In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


More Complex JOIN's and Partition'ing

In [ ]:
%%bigquery --project august-bucksaw-330517
select p.payment_id, p.amount, p.customer_id, p.rental_id, i.film_id, f.title, f.rating
from `jrjames83-1171.sampledata.payments` p
join `jrjames83-1171.sampledata.rental` r on p.rental_id=r.rental_id
join `jrjames83-1171.sampledata.inventory` i on r.inventory_id = i.inventory_id
join `jrjames83-1171.sampledata.film` f on i.film_id = f.film_id
limit 5

,payment_id,amount,customer_id,rental_id,film_id,title,rating
0,19462,2.99,256,1555,906,Tramp Others,PG
1,19464,4.99,256,1973,803,Slacker Liaisons,R
2,31791,0.99,256,4182,677,Pianist Outfield,NC-17
3,31792,2.99,256,5179,657,Paradise Sabrina,PG-13
4,31795,2.99,256,9424,982,Women Dorado,R


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select p.payment_id, p.amount, p.customer_id, p.rental_id, i.film_id, f.title, f.rating
from `jrjames83-1171.sampledata.payments` p
join `jrjames83-1171.sampledata.rental` r on p.rental_id=r.rental_id
join `jrjames83-1171.sampledata.inventory` i on r.inventory_id = i.inventory_id
join `jrjames83-1171.sampledata.film` f on i.film_id = f.film_id)

select bt.customer_id, bt.rating, sum(bt.amount) rental_revenue 
from base_table bt
group by 1,2
order by 1
limit 5

,customer_id,rating,rental_revenue
0,1,PG-13,24.94
1,1,PG,28.92
2,1,NC-17,32.92
3,1,G,11.97
4,1,R,15.95


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select p.payment_id, p.amount, p.customer_id, p.rental_id, i.film_id, f.title, f.rating
from `jrjames83-1171.sampledata.payments` p
join `jrjames83-1171.sampledata.rental` r on p.rental_id=r.rental_id
join `jrjames83-1171.sampledata.inventory` i on r.inventory_id = i.inventory_id
join `jrjames83-1171.sampledata.film` f on i.film_id = f.film_id),

summary_table as (select bt.customer_id, bt.rating, sum(bt.amount) rental_revenue 
from base_table bt
group by 1,2
order by 1)

select * from (select 
    st.*, row_number() over (partition by st.customer_id order by st.rental_revenue desc) rating_customer_revenue
    from summary_table st
    order by st.customer_id) t
where t.rating_customer_revenue < 3
limit 5

,customer_id,rating,rental_revenue,rating_customer_revenue
0,1,NC-17,32.92,1
1,1,PG,28.92,2
2,2,PG,30.94,2
3,2,G,30.96,1
4,3,R,66.89,1


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select p.payment_id, p.amount, p.payment_date, p.customer_id, p.rental_id, i.film_id, f.title, f.rating
from `jrjames83-1171.sampledata.payments` p
join `jrjames83-1171.sampledata.rental` r on p.rental_id=r.rental_id
join `jrjames83-1171.sampledata.inventory` i on r.inventory_id = i.inventory_id
join `jrjames83-1171.sampledata.film` f on i.film_id = f.film_id),

summary_table as (select bt.customer_id, bt.rating, extract(month from bt.payment_date) month, sum(bt.amount) rental_revenue 
from base_table bt
group by 1,2,3
order by 1)

select * from (select 
    st.*, row_number() over (partition by st.customer_id, st.month order by st.rental_revenue desc) rating_customer_revenue_month
    from summary_table st
    order by st.customer_id) t
where t.rating_customer_revenue_month < 3
limit 5

,customer_id,rating,month,rental_revenue,rating_customer_revenue_month
0,1,NC-17,3,15.96,1
1,1,PG,2,15.98,1
2,1,PG,3,6.96,2
3,1,NC-17,4,12.97,2
4,1,PG-13,4,12.98,1
